In [62]:

import numpy as np
import seaborn as sns
import pandas as pd

<font size='4' face='Times new Roman'>
    Dit worden de US Factors
    <font>

In [64]:
file_url = 'https://raw.githubusercontent.com/Sebasleen/Seminargroup/Seminar/US%20Factors/anomalies.dta'

# Read the Stata file content
Anomalies = pd.read_stata(file_url)

# Display unique values in the 'anomaly' column
print(Anomalies['anomaly'].unique())

# Display the DataFrame
print(Anomalies)

#Hier heb ik de global factors verwijderd
column_name = 'anomaly'
values_to_drop = ['glbab', 'glcma', 'glhml', 'glqmj', 'glrmw', 'glsmb', 'glumd']
mask = Anomalies[column_name].isin(values_to_drop)
Anomalies_US = Anomalies[~mask]

#Dit zijn dus alle 15 factors die alleen bij de US horen. 
print(Anomalies_US)





['ac' 'bab' 'cfp' 'cma' 'ep' 'hml' 'liq' 'ltrev' 'nsi' 'qmj' 'rmw' 'rvar'
 'smb' 'strev' 'umd' 'glbab' 'glcma' 'glhml' 'glqmj' 'glrmw' 'glsmb'
 'glumd']
       year  month anomaly    ret   time  global
0      1963      7      ac  2.170   42.0     0.0
1      1963      8      ac -0.197   43.0     0.0
2      1963      9      ac  0.600   44.0     0.0
3      1963     10      ac  6.463   45.0     0.0
4      1963     11      ac -2.260   46.0     0.0
...     ...    ...     ...    ...    ...     ...
12638  2019      8   glumd  2.990  715.0     1.0
12639  2019      9   glumd -3.260  716.0     1.0
12640  2019     10   glumd -0.940  717.0     1.0
12641  2019     11   glumd  0.000  718.0     1.0
12642  2019     12   glumd  0.740  719.0     1.0

[12643 rows x 6 columns]
       year  month anomaly    ret   time  global
0      1963      7      ac  2.170   42.0     0.0
1      1963      8      ac -0.197   43.0     0.0
2      1963      9      ac  0.600   44.0     0.0
3      1963     10      ac  6.463   4

<font size='4' face='Times new Roman'>    

Voor de duidelijkheid, dit zijn de US factors:

- **-ac**: Accruals
- **-bab**: Betting against Beta
- **-cfp**: Cash-flow to price
- **-cma**: Investment (Conservative minus aggressive)
- **-ep**: Earnings to price
- **-hml**: Value
- **-liq**: Liquidity
- **-ltrev**: Long-term reversal
- **-nsi**: Net share issue
- **-qmj**: Quality minus junk
- **-rmw**: Profitability
- **-rvar**: Residual Variance
- **-smb**: Size
- **-strev**: Short-term reversals
- **-umd**: Momentum

En dit zijn de global factors die ik zojuist heb gedropt:

- **glbab**: Betting against beta
- **glcma**: Investment (Conservative minus aggressive)
- **glhml**: Value (high minus low)
- **glqmj**: Quality minus junk
- **glrmw**: Profitability (robust minus weak)
- **glumd**: Momentum (up minus down)
- **glsmb**: Size (small minus big)

</font>


In [86]:
#T-value krijg ik niet goed, de andere 2 wel. 

import pandas as pd
import numpy as np
from scipy.stats import t

# Calculate mean, standard deviation, standard error, count, and t-statistic for each anomaly
table_data = []
for anomaly in Anomalies_US['anomaly'].unique():
    anomaly_data = Anomalies_US[Anomalies_US['anomaly'] == anomaly]['ret']
    mean = anomaly_data.mean() * 12  # Annualized mean
    std_dev = anomaly_data.std() * np.sqrt(12)  # Annualized standard deviation
    n = anomaly_data.count()  # Number of observations
    stderr = std_dev / np.sqrt(n)  # Standard error
    t_stat = mean / stderr
    p_value = t.cdf(-np.abs(t_stat), df=n-1) * 2  # two-tailed test
    table_data.append([anomaly, mean, std_dev, t_stat, p_value, n])

# Create DataFrame for the table
table_df = pd.DataFrame(table_data, columns=['Anomaly', 'Annualized Mean', 'Annualized Standard Deviation', 'T-statistic', 'P-value', 'Count'])

# Display the table sorted by start date
table_df.sort_values(by=['Anomaly'], inplace=True)

print(table_df)


   Anomaly  Annualized Mean  Annualized Standard Deviation  T-statistic  \
0       ac         2.812796                       6.633251    11.041466   
1      bab         9.799457                      11.243013    22.695207   
2      cfp         3.379115                       8.643392    10.179668   
3      cma         3.290974                       6.900146    12.418839   
4       ep         3.482212                       8.887625    10.201977   
5      hml         3.649381                       9.727490     9.768620   
6      liq         4.444018                      11.563227     9.600395   
7    ltrev         2.489911                       8.668542     7.479158   
8      nsi         2.756519                       8.208919     8.743591   
9      qmj         4.596994                       7.738000    15.468922   
10     rmw         3.106195                       7.469543    10.828030   
11    rvar         1.551858                      17.259279     2.341231   
12     smb         2.7332

In [8]:
print(Anomalies_US)

       year  month anomaly    ret   time  global
0      1963      7      ac  2.170   42.0     0.0
1      1963      8      ac -0.197   43.0     0.0
2      1963      9      ac  0.600   44.0     0.0
3      1963     10      ac  6.463   45.0     0.0
4      1963     11      ac -2.260   46.0     0.0
...     ...    ...     ...    ...    ...     ...
10111  2019      8     umd  7.600  715.0     0.0
10112  2019      9     umd -6.850  716.0     0.0
10113  2019     10     umd  0.240  717.0     0.0
10114  2019     11     umd -2.620  718.0     0.0
10115  2019     12     umd -2.130  719.0     0.0

[10116 rows x 6 columns]


In [72]:
# Get all unique values in the 'anomaly' column
unique_anomalies = Anomalies['anomaly'].unique()

# Create a dictionary to store the separate DataFrames
anomaly_dataframes = {}

# Iterate over each unique anomaly and filter the DataFrame
for anomaly in unique_anomalies:
    # Filter the DataFrame for the current anomaly
    anomaly_df = Anomalies[Anomalies['anomaly'] == anomaly]
    
    
    # Store the filtered DataFrame in the dictionary
    anomaly_dataframes[anomaly] = anomaly_df

# Now anomaly_dataframes dictionary contains separate DataFrames for each unique anomaly
# You can access each DataFrame using the anomaly name as the key

anomaly_dataframes['bab']


,year,month,anomaly,ret,time,global
678,1963,7,bab,0.812285,42.0,0.0
679,1963,8,bab,-1.215830,43.0,0.0
680,1963,9,bab,1.068651,44.0,0.0
681,1963,10,bab,-1.875605,45.0,0.0
682,1963,11,bab,-0.519820,46.0,0.0
...,...,...,...,...,...,...
1351,2019,8,bab,2.400757,715.0,0.0
1352,2019,9,bab,-1.162819,716.0,0.0
1353,2019,10,bab,-0.410805,717.0,0.0
1354,2019,11,bab,-2.939773,718.0,0.0


In [58]:
#Hier staat wat hij in stata doet, en waarom de ret_mean telkens anders was dan in het paper staat.
#blijkbaar moet je dat nog x12 doen om het te annualizen en dan kom je bij de percentages uit die hij in het paper
#heeft staan. 

import pandas as pd
import numpy as np

# Load the data
data = pd.read_stata("anomalies.dta")

# Calculate statistics by 'global' and 'anomaly'
summary_stats = data.groupby(['global', 'anomaly']).agg({'ret': ['mean', 'std', 'count']}).reset_index()

# Rename columns
summary_stats.columns = ['global', 'anomaly', 'ret_mean', 'ret_sd', 'ret_count']

# Calculate additional statistics
summary_stats['ret_semean'] = summary_stats['ret_sd'] / np.sqrt(summary_stats['ret_count'])
summary_stats['ret'] = summary_stats['ret_mean'] * 12
summary_stats['sd'] = summary_stats['ret_sd'] * np.sqrt(12)
summary_stats['tstat'] = summary_stats['ret'] / summary_stats['ret_semean']

# Format columns
summary_stats[['ret', 'sd', 'tstat']] = summary_stats[['ret', 'sd', 'tstat']].applymap("{:.2f}".format)

# Print the DataFrame
print(summary_stats)


    global anomaly  ret_mean    ret_sd  ret_count  ret_semean   ret     sd  \
0      0.0      ac  0.234400  1.914855        678    0.073540  2.81   6.63   
1      0.0     bab  0.816621  3.245579        678    0.124646  9.80  11.24   
2      0.0     cfp  0.281593  2.495132        678    0.095825  3.38   8.64   
3      0.0     cma  0.274248  1.991900        678    0.076498  3.29   6.90   
4      0.0      ep  0.290184  2.565636        678    0.098533  3.48   8.89   
5      0.0     hml  0.304115  2.808084        678    0.107844  3.65   9.73   
6      0.0     liq  0.370335  3.338016        624    0.133628  4.44  11.56   
7      0.0   ltrev  0.207493  2.502393        678    0.096104  2.49   8.67   
8      0.0     nsi  0.229710  2.369711        678    0.091008  2.76   8.21   
9      0.0     qmj  0.383083  2.233768        678    0.085787  4.60   7.74   
10     0.0     rmw  0.258850  2.156272        678    0.082811  3.11   7.47   
11     0.0    rvar  0.129322  4.982324        678    0.191345  1

/var/folders/pf/g9gmd9ss1kb0ffhstq1zx8xw0000gn/T/ipykernel_54801/3640921379.py:20: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  summary_stats[['ret', 'sd', 'tstat']] = summary_stats[['ret', 'sd', 'tstat']].applymap("{:.2f}".format)
